In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import tensorflow as tf

# needed to avoid a tf error
try:
    gpus= tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(gpus[0], True)
except:
    pass

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf
import numpy as np

# Boston Model

In [13]:
# loading models that we prepared earlier
X_train = np.load(os.path.join('data', 'boston', 'boston_train_vectors.npy'), allow_pickle=True)
X_test = np.load(os.path.join('data', 'boston', 'boston_test_vectors.npy'), allow_pickle=True)
y_train = np.load(os.path.join('data', 'boston', 'boston_train_labels.npy'), allow_pickle=True)
y_test = np.load(os.path.join('data', 'boston', 'boston_test_labels.npy'), allow_pickle=True)

In [14]:
X_train.shape, y_train.shape

((404, 13), (404,))

In [15]:
# setting seed to get reproducible results
tf.random.set_seed(42)
np.random.seed(42)

# building a small model as an experiment
def build_model():
    model = Sequential()
    model.add(Dense(13, activation='sigmoid'))
    model.add(Dense(50, activation='sigmoid'))
    model.add(Dense(50, activation='sigmoid'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='sgd', metrics=['mae'])
    return model

In [16]:
# early stopping is not used in the current implementation, but we plan to use it in the final model
cb = [EarlyStopping(monitor="val_mae", min_delta=0.01, patience=2, verbose=1, 
                    mode="auto", baseline=None, restore_best_weights=True)]

In [17]:
#model.fit(X_train, y_train, batch_size=1, epochs=20, validation_split=0.1, callbacks=cb)

# Using the custom training loop

In [18]:
# importing our custom loop
from loop import TrainingLoop
# importing our batch selection algorithms
from batch_selection import windowed_batch_selector, sorting_batch_selector

In [19]:
# using SGD oprimizer for training
optimizer = tf.keras.optimizers.SGD()

# MSE loss function for this regression task
loss_function = tf.keras.losses.MeanSquaredError()

batch_size = 8
epochs = 20

# using MAE as our secondary metric
train_metrics = tf.keras.metrics.MeanAbsoluteError()
val_metrics = tf.keras.metrics.MeanAbsoluteError()

In [20]:
def train(model, X_train, y_train, batch_selector, epochs):
    selectors = {windowed_batch_selector: 'windowed', sorting_batch_selector: 'sorting', None: 'original'}
    print('\n\n'+selectors[batch_selector]+'\n')
    # defining the training class
    training = TrainingLoop(model, X_train, y_train, 
        loss_function, 
        optimizer, 
        train_metrics, 
        val_metrics, 
        validation_split=0.1, 
        batch_size=batch_size,
        batch_selection=batch_selector,
        log_file = os.path.join('logs',selectors[batch_selector],'boston_houses.csv')
    )
    # training the model
    training.train(epochs)

In [21]:
def evaluate(model, X_test, y_test):
    res = model.evaluate(X_test, y_test)
    print(np.sqrt(res[0]), res[1])

In [22]:
# Train and evaluate models with different batch selection algorithms.
for batch_selector in [None, windowed_batch_selector, sorting_batch_selector]:
    # Set random seed so the comparison of different solutions won't be affected by it.
    tf.random.set_seed(42)
    np.random.seed(42)
    
    model = build_model()
    train(model, X_train, y_train, batch_selector=batch_selector, epochs=epochs)
    evaluate(model, X_test, y_test)

	  0% | 0/45 [00:00<?, ?it/s]

original


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/20	Loss: 79.287	Metrics: 7.9209: 	Validation metrics: 6.1495: 	100% | 45/45 [00:03<00:00, 14.69it/s]
Epoch 2/20	Loss: 41.868	Metrics: 6.5750: 	Validation metrics: 4.6793: 	100% | 45/45 [00:00<00:00, 85.07it/s]
Epoch 3/20	Loss: 20.744	Metrics: 4.5591: 	Validation metrics: 3.4657: 	100% | 45/45 [00:00<00:00, 75.76it/s]
Epoch 4/20	Loss: 12.836	Metrics: 3.8331: 	Validation metrics: 3.3674: 	100% | 45/45 [00:00<00:00, 71.20it/s]
Epoch 5/20	Loss: 10.202	Metrics

ValueError: in user code:

    d:\BME\5_felev\deep_learning\HF\Deep-Learning-Shuffle-Algorithm\batch_selection.py:9 calc_loss  *
        logits = model(x_train, training=False)
    D:\progs\python-3.8.6-amd64\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:985 __call__  **
        outputs = call_fn(inputs, *args, **kwargs)
    D:\progs\python-3.8.6-amd64\lib\site-packages\tensorflow\python\keras\engine\sequential.py:386 call
        outputs = layer(inputs, **kwargs)
    D:\progs\python-3.8.6-amd64\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:982 __call__
        self._maybe_build(inputs)
    D:\progs\python-3.8.6-amd64\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:2643 _maybe_build
        self.build(input_shapes)  # pylint:disable=not-callable
    D:\progs\python-3.8.6-amd64\lib\site-packages\tensorflow\python\keras\layers\core.py:1171 build
        self.kernel = self.add_weight(
    D:\progs\python-3.8.6-amd64\lib\site-packages\tensor